In [1]:
import os
import sys
import subprocess
import datetime as dt
import itertools
import numpy as np
import xarray as xr
from dask.distributed import Client, LocalCluster
import dask.array as da

In [2]:
cluster = LocalCluster(n_workers=4,threads_per_worker=1,dashboard_address='localhost:54924')
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:54924/status,
Dashboard: http://127.0.0.1:54924/status,Workers: 4
Total threads: 4,Total memory: 0.97 TiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:32843,Workers: 4
Dashboard: http://127.0.0.1:54924/status,Total threads: 4
Started: Just now,Total memory: 0.97 TiB
Comm: tcp://127.0.0.1:42693,Total threads: 1
Dashboard: http://127.0.0.1:37031/status,Memory: 248.04 GiB
Nanny: tcp://127.0.0.1:47095,


In [3]:
mdir='/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/data/predictions/cansipsv3_daily/CanESM5'
fnameCanESMjoined=lambda mdir, yyyy, mm, dd, hh:f"{mdir}/joined/cwao_CanESM5.1p1bc-v20240611_hindcast_S{yyyy:04}{mm:02}{dd:02}{hh:02}_ocean_6hr_surface_tso.nc"
fnameCanESMdaily=lambda mdir, yyyy, mm, dd, hh:f"{mdir}/joined/cwao_CanESM5.1p1bc-v20240611_hindcast_S{yyyy:04}{mm:02}{dd:02}{hh:02}_ocean_1d_surface_tso.nc"
yyyy=1998
mm=2
dd=1
hh=0

In [4]:
215/5

43.0

In [5]:
fin=fnameCanESMjoined(mdir,yyyy,mm,dd,hh)
fout=fnameCanESMdaily(mdir,yyyy,mm,dd,hh)
print(fout,flush=True)
ff=xr.open_mfdataset(fin,decode_times=False,parallel=True,)
ff0=ff.chunk(chunks={'record':-1,'leadtime':4,'lat':-1,'lon':-1})

/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/data/predictions/cansipsv3_daily/CanESM5/joined/cwao_CanESM5.1p1bc-v20240611_hindcast_S1998020100_ocean_1d_surface_tso.nc


In [6]:
ff0.tso

<xarray.DataArray 'tso' (record: 20, leadtime: 860, lat: 180, lon: 360)> Size: 9GB
dask.array<rechunk-merge, shape=(20, 860, 180, 360), dtype=float64, chunksize=(20, 4, 180, 360), chunktype=numpy.ndarray>
Coordinates:
  * lat          (lat) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
  * leadtime     (leadtime) float64 7kB 6.0 12.0 18.0 ... 5.154e+03 5.16e+03
  * lon          (lon) float64 3kB 0.5 1.5 2.5 3.5 ... 356.5 357.5 358.5 359.5
    realization  |S31 31B ...
    reftime      float64 8B ...
    time         (leadtime) float64 7kB dask.array<chunksize=(4,), meta=np.ndarray>
Dimensions without coordinates: record
Attributes:
    cell_methods:   leadtime: point
    grid_mapping:   hcrs
    long_name:      Sea Surface Temperature
    standard_name:  sea_surface_temperature
    units:          K

In [7]:
ff2=ff0.drop_vars(['realization','hcrs']).rename({'record':'r'})
ff3=ff2.coarsen(leadtime=4).mean()
#ff3.tso.assign_attrs({'postprocess':'daily time average, [(6,12,18,24),...]'})
ff3.tso

<xarray.DataArray 'tso' (r: 20, leadtime: 215, lat: 180, lon: 360)> Size: 2GB
dask.array<mean_agg-aggregate, shape=(20, 215, 180, 360), dtype=float64, chunksize=(20, 1, 180, 360), chunktype=numpy.ndarray>
Coordinates:
  * lat       (lat) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * leadtime  (leadtime) float64 2kB 15.0 39.0 63.0 ... 5.127e+03 5.151e+03
  * lon       (lon) float64 3kB 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
    reftime   float64 8B 0.0
    time      (leadtime) float64 2kB dask.array<chunksize=(1,), meta=np.ndarray>
Dimensions without coordinates: r
Attributes:
    cell_methods:   leadtime: point
    grid_mapping:   hcrs
    long_name:      Sea Surface Temperature
    standard_name:  sea_surface_temperature
    units:          K

In [8]:
%%time
ff3.to_netcdf(fout,mode='w')

CPU times: user 3min 9s, sys: 19.7 s, total: 3min 29s
Wall time: 1h 40s


In [9]:
del ff3; del ff2;
ff.close();
del ff